# Cleaning

In [51]:
import pandas as pd
df = pd.read_csv('./data.csv')

In [52]:
df

features__id  features__geometry__coordinates__001  \
0         7982642                             73.856285   
1         1880291                             73.855774   
2         7989636                             73.855408   
3         8119763                             73.855835   
4         4982515                             73.855301   
..            ...                                   ...   
495       5167813                             73.860718   
496       7989644                             73.869949   
497       1880659                             73.849441   
498       1880658                             73.848267   
499       5102378                             73.865364   

     features__geometry__coordinates__002  features__properties__name  \
0                               18.519789      Navagraha Shani Mandir   
1                               18.519611                         NaN   
2                               18.520498                         NaN   
3                               18.519409                         NaN   
4                               18.520477  Shaniwar Wada Amphitheatre   
..                                    ...                         ...   
495                             18.754732                         NaN   
496                             18.754507                         NaN   
497                             18.755674                         NaN   
498                             18.756159                         NaN   
499                             18.756971  Bolhai mata mandir, chakan   

                           features__properties__kinds  
0            religion,hindu_temples,interesting_places  
1    architecture,historic_architecture,interesting...  
2            religion,hindu_temples,interesting_places  
3    architecture,historic_architecture,interesting...  
4    architecture,historic_architecture,fortificati...  
..                                                 ...  
495          religion,hindu_temples,interesting_places  
496          religion,hindu_temples,interesting_places  
497          religion,hindu_temples,interesting_places  
498          religion,hindu_temples,interesting_places  
499          religion,other_temples,interesting_places  

[500 rows x 5 columns]

In [66]:
df.isnull().sum()

features__id                             0
features__geometry__coordinates__001     0
features__geometry__coordinates__002     0
features__properties__name              79
features__properties__kinds              0
dtype: int64

In [67]:
new_df = df.dropna()
new_df = new_df[~new_df['features__properties__name'].str.contains('\d', na=False)]

In [68]:
new_df.to_csv("./new_data.csv")

# CBR

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [70]:
ds = pd.read_csv("./new_data.csv")

In [71]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['features__properties__kinds'])

In [72]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}

In [73]:
for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], ds['features__id'][i]) for i in similar_indices]

    results[row['features__id']] = similar_items[1:]
    
print('done!')

done!


In [74]:
def item(id):
    return ds.loc[ds['features__id'] == id]['features__properties__name'].tolist()[0]

In [75]:
def recommend(item_id, num):
    print("Recommending " + str(num) + " places similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

In [77]:
recommend(item_id=1880355, num=10)

Recommending 10 places similar to COEP Auditorium...
-------
Recommended: Ankushrao Landge Natyagruha (score:0.9999999999999999)
Recommended: Dhanwantari Auditorium (score:0.9999999999999999)
Recommended: Amphitheatre (score:0.9999999999999999)
Recommended: Amphitheater (score:0.9999999999999999)
Recommended: Nehru Memorial Hall (score:0.9999999999999999)
Recommended: Mini Auditorium COEP (score:0.9999999999999999)
Recommended: COEP Auditorium (score:0.9999999999999999)
Recommended: Auditorium (score:0.9999999999999999)
Recommended: Pandit Bhimsen Joshi Kalamandir (score:0.9999999999999999)
Recommended: Bharat Naatya Mandir (score:0.9999999999999999)
